<a href="https://colab.research.google.com/github/fedeflowers/AlphaEvolveTryout/blob/main/Autolavaggio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import os

root_drive_path = '/content/drive/MyDrive'
VERSION = "v3"


# Define the relative paths provided by the user
path1_relative = 'tex_cars/CatalogatedImagesAI/GenerateAI/pista1/OpenHood'
path2_relative = 'tex_cars/CatalogatedImagesAI/GenerateAI/pista1/Original'
base_model_dir = os.path.join(root_drive_path, 'tex_cars')

mobilenet_save_dir = os.path.join(base_model_dir, 'mobilenet', VERSION)
yolo_save_dir = os.path.join(base_model_dir, 'yolo', VERSION)

os.makedirs(mobilenet_save_dir, exist_ok=True)
os.makedirs(yolo_save_dir, exist_ok=True)

print(f"Models will be saved to:\n - {mobilenet_save_dir}\n - {yolo_save_dir}")


# Construct full paths
full_path1 = os.path.join(root_drive_path, path1_relative)
full_path2 = os.path.join(root_drive_path, path2_relative)

image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')

def list_image_files(directory_path):
    images = []
    if os.path.exists(directory_path):
        print(f"Listing images in: {directory_path}")
        for filename in os.listdir(directory_path):
            if filename.lower().endswith(image_extensions):
                images.append(os.path.join(directory_path, filename))
    else:
        print(f"Directory not found: {directory_path}")
    return images

print("\n--- Images from OpenHood ---")
open_hood_images = list_image_files(full_path1)
# for img in open_hood_images:
#     print(img)

print("\n--- Images from Original ---")
original_images = list_image_files(full_path2)
# for img in original_images:
#     print(img)


Models will be saved to:
 - /content/drive/MyDrive/tex_cars/mobilenet/v3
 - /content/drive/MyDrive/tex_cars/yolo/v3

--- Images from OpenHood ---
Listing images in: /content/drive/MyDrive/tex_cars/CatalogatedImagesAI/GenerateAI/pista1/OpenHood

--- Images from Original ---
Listing images in: /content/drive/MyDrive/tex_cars/CatalogatedImagesAI/GenerateAI/pista1/Original


In [13]:
len(original_images), len(open_hood_images)

(306, 306)

In [14]:
import shutil
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Get list of filenames (assuming filenames are identical in both folders)
# We use the 'Original' folder as the source of truth for unique cars
original_filenames = [os.path.basename(x) for x in original_images]

# 2. Split the FILENAMES, not the images
train_names, val_names = train_test_split(original_filenames, test_size=0.2, random_state=42)

def build_dataset(filenames, original_paths, open_paths):
    data = []
    # Create a lookup for open_paths for speed
    open_map = {os.path.basename(p): p for p in open_paths}
    original_map = {os.path.basename(p): p for p in original_paths}

    for name in filenames:
        # Add the 'Closed' version (Label 0)
        if name in original_map:
            data.append({
                'filepath': original_map[name],
                'is_hood_open': 0,
                'car_id': name # Useful for debugging
            })

        # Add the 'Open' version (Label 1)
        # Only add if it exists in the Open folder
        if name in open_map:
            data.append({
                'filepath': open_map[name],
                'is_hood_open': 1,
                'car_id': name
            })
    return pd.DataFrame(data)

# 3. Build the DataFrames
train_df = build_dataset(train_names, original_images, open_hood_images)
val_df = build_dataset(val_names, original_images, open_hood_images)

# Convert for Keras
train_df['is_hood_open_str'] = train_df['is_hood_open'].astype(str)
val_df['is_hood_open_str'] = val_df['is_hood_open'].astype(str)

print(f"Training Set: {len(train_df)} images")
print(f"Validation Set: {len(val_df)} images")

Training Set: 488 images
Validation Set: 124 images


In [15]:
val_df.head(10)['filepath'][0]

'/content/drive/MyDrive/tex_cars/CatalogatedImagesAI/GenerateAI/pista1/Original/pista1_091125_135823.png'

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, models
# import os
# from tensorflow.keras.applications.mobilenet_v3 import preprocess_input

# # --- A. Preprocessing & Loading ---
# BATCH_SIZE = 32
# IMG_SIZE = (224, 224)

# train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
#     preprocessing_function=preprocess_input,
#     rotation_range=20
# )
# val_gen = tf.keras.preprocessing.image.ImageDataGenerator(
#     preprocessing_function=preprocess_input
# )

# train_data = train_gen.flow_from_dataframe(
#     train_df, x_col='filepath', y_col='is_hood_open_str',
#     target_size=IMG_SIZE, class_mode='binary', batch_size=BATCH_SIZE
# )
# val_data = val_gen.flow_from_dataframe(
#     val_df, x_col='filepath', y_col='is_hood_open_str',
#     target_size=IMG_SIZE, class_mode='binary', batch_size=BATCH_SIZE
# )

# # --- B. Model Definition ---
# base_model = tf.keras.applications.MobileNetV3Small(
#     input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet'
# )
# base_model.trainable = False

# model = models.Sequential([
#     base_model,
#     layers.GlobalAveragePooling2D(),
#     layers.Dropout(0.2),
#     layers.Dense(1, activation='sigmoid')
# ])

# model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
#               loss='binary_crossentropy',
#               metrics=['accuracy', tf.keras.metrics.Recall(name='recall')])

# # --- C. Training ---
# # Class 0 = Closed (Normal), Class 1 = Open (Fault)
# # Weight 7.0 on Class 1 forces the model to prioritize detecting Open hoods (Recall)
# class_weights = {0: 1.0, 1: 1.0}
# model.fit(train_data, validation_data=val_data, epochs=100, class_weight=class_weights)

# # --- D. Export to TFLite ---
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_model = converter.convert()

# mn_path = os.path.join(mobilenet_save_dir, 'hood_classifier.tflite')
# with open(mn_path, 'wb') as f:
#     f.write(tflite_model)
# print(f"MobileNet Saved: {mn_path}")

Found 488 validated image filenames belonging to 2 classes.
Found 124 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.5377 - loss: 0.7925 - recall: 0.3572 - val_accuracy: 0.5726 - val_loss: 0.6636 - val_recall: 0.6129
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.5330 - loss: 0.7451 - recall: 0.4138 - val_accuracy: 0.6129 - val_loss: 0.6638 - val_recall: 0.8387
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.6123 - loss: 0.6763 - recall: 0.5649 - val_accuracy: 0.6048 - val_loss: 0.6603 - val_recall: 0.9032
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.5623 - loss: 0.6959 - recall: 0.5554 - val_accuracy: 0.6452 - val_loss: 0.6453 - val_recall: 0.9032
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.5667 - loss: 0.7191 - recall: 0.5309 - val_accuracy: 0.6532 - val_loss: 0.6391 - val_recall: 0.9355
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.5972 - loss: 0.6776 - recall: 0.6004 - val_accuracy: 0.6613 - val_loss: 0.6239 - val_recall: 0.9194
Epoch 7/10

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
# from sklearn.metrics import accuracy_score, classification_report

# # 1. Load TFLite Model
# interpreter = tf.lite.Interpreter(model_path=mn_path)
# interpreter.allocate_tensors()

# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()
# input_index = input_details[0]['index']
# output_index = output_details[0]['index']

# print("Model loaded. Starting inference...")

# predictions = []
# actuals = []

# # 2. Iterate through Validation Data
# # Assuming val_df has columns 'filepath' and 'is_hood_open_str' (or numeric label)
# for index, row in val_df.iterrows():
#     img_path = row['filepath']

#     # Load and Preprocess (Must match training exactly)
#     try:
#         img = image.load_img(img_path, target_size=(224, 224))
#         x = image.img_to_array(img)
#         x = preprocess_input(x) # MobileNetV3 specific preprocessing
#         x = np.expand_dims(x, axis=0) # Add batch dimension

#         # Run Inference
#         interpreter.set_tensor(input_index, x)
#         interpreter.invoke()
#         output_data = interpreter.get_tensor(output_index)

#         # Binary thresholding (Sigmoid output)
#         pred_label = 1 if output_data[0][0] > 0.5 else 0
#         predictions.append(pred_label)

#         # Get actual label (handle string or int)
#         actual = row['is_hood_open_str']
#         actual_label = 1 if str(actual) == '1' or str(actual).lower() == 'open' else 0
#         actuals.append(actual_label)

#     except Exception as e:
#         print(f"Error processing {img_path}: {e}")

# # 3. Metrics
# print(f"\nAccuracy: {accuracy_score(actuals, predictions):.4f}")
# print("\nClassification Report:\n")
# print(classification_report(actuals, predictions, target_names=['Closed', 'Open']))

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Model loaded. Starting inference...

Accuracy: 0.9677

Classification Report:

              precision    recall  f1-score   support

      Closed       1.00      0.94      0.97        62
        Open       0.94      1.00      0.97        62

    accuracy                           0.97       124
   macro avg       0.97      0.97      0.97       124
weighted avg       0.97      0.97      0.97       124



# YOLO

In [ ]:
import os
import shutil
import subprocess
import glob
import numpy as np
from sklearn.metrics import classification_report, precision_recall_fscore_support

print("\n--- Training YOLOv11 with Live Precision/Recall ---")

# 1. Install Ultralytics
try:
    from ultralytics import YOLO
except ImportError:
    subprocess.check_call(["pip", "install", "ultralytics"])
    from ultralytics import YOLO

# 2. Prepare Directory Structure
yolo_data_dir = '/content/yolo_dataset'
val_path = os.path.join(yolo_data_dir, 'val') # Define globally for callback

if os.path.exists(yolo_data_dir):
    shutil.rmtree(yolo_data_dir)

def get_class_folder(label):
    s_label = str(label).lower()
    if s_label == '1' or s_label == 'open':
        return 'Open'
    return 'Closed'

# Create train and val directories
train_dir_yolo = os.path.join(yolo_data_dir, 'train')
val_dir_yolo = os.path.join(yolo_data_dir, 'val')
os.makedirs(train_dir_yolo, exist_ok=True)
os.makedirs(val_dir_yolo, exist_ok=True)

for subset_name, df_part in [('train', train_df), ('val', val_df)]:
    for _, row in df_part.iterrows():
        folder_name = get_class_folder(row['is_hood_open_str'])
        # Use the specific subset directory (train_dir_yolo or val_dir_yolo)
        dest_dir = os.path.join(yolo_data_dir, subset_name, folder_name)
        os.makedirs(dest_dir, exist_ok=True)
        shutil.copy(row['filepath'], os.path.join(dest_dir, os.path.basename(row['filepath'])))


# 3. Initialize Model
model_yolo = YOLO("yolo11m-cls.pt")

# 4. Define Custom Callback
def on_train_epoch_end(trainer):
    """
    Custom callback to calculate P/R at the end of every epoch.
    """
    print(f"\n[Epoch {trainer.epoch + 1}] Calculating Precision/Recall...")

    # FIX: Do NOT use trainer.model directly as it breaks the gradient graph.
    # Instead, load the latest saved weights from disk into a fresh instance.
    weight_path = os.path.join(trainer.save_dir, 'weights', 'last.pt')

    if not os.path.exists(weight_path):
        print("   >> No saved weights found yet. Skipping metrics.")
        return

    # Load temp model for safe inference
    temp_model = YOLO(weight_path)

    # Gather validation images
    val_images = glob.glob(os.path.join(val_path, '**', '*'), recursive=True)
    val_images = [f for f in val_images if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp'))]

    if not val_images:
        print(f"⚠️ Warning: No images found in {val_path}. Skipping metrics.")
        return

    # Run inference
    results = temp_model.predict(source=val_images, stream=True, verbose=False)

    y_true = []
    y_pred = []

    for r in results:
        # Extract true label from folder structure
        folder = os.path.basename(os.path.dirname(r.path))
        true_label = 1 if folder.lower() == 'open' else 0
        y_true.append(true_label)

        # Extract predicted label
        pred_class_name = temp_model.names[r.probs.top1]
        pred_label = 1 if pred_class_name.lower() == 'open' else 0
        y_pred.append(pred_label)

    # Calculate Metrics
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='binary', zero_division=0
    )

    print(f"   >> Precision (Open): {precision:.4f}")
    print(f"   >> Recall    (Open): {recall:.4f}")
    print(f"   >> F1-Score  (Open): {f1:.4f}\n")

    # Clean up to save memory
    del temp_model

# 5. Register Callback and Train
model_yolo.add_callback("on_train_epoch_end", on_train_epoch_end)

# Define run parameters
project_path = '/content/yolo_runs'
run_name = 'hood_run'

model_yolo.train(
    data=yolo_data_dir,
    epochs=3,
    imgsz=224,
    project='/content/yolo_runs',
    name='hood_run',
    # Augmentations
    degrees=180,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    scale=0.5,
    erasing=0.4,
    mixup=0.1
)

# 6. ROBUST EXPORT & SAVE
print("\n--- Locating Latest Run and Exporting ---")

# 1. Find the latest run folder (e.g., hood_run, hood_run2, hood_run3)
# We search for all folders starting with 'hood_run' in the project directory
possible_runs = glob.glob(os.path.join(project_path, f"{run_name}*"))

if possible_runs:
    # Select the folder with the most recent modification time
    latest_run_dir = max(possible_runs, key=os.path.getmtime)
    print(f"📍 Detected latest run: {latest_run_dir}")

    # Define path to best.pt in this specific run
    best_weight_path = os.path.join(latest_run_dir, 'weights', 'best.pt')
else:
    print(f"❌ Error: No folders found starting with {run_name}")
    exit()

# 2. Proceed with Export if weights exist
if os.path.exists(best_weight_path):
    print(f"Found best model weights: {best_weight_path}")

    # Save .pt to Drive
    pt_dest = os.path.join(yolo_save_dir, 'yolo_hood_best.pt')
    shutil.copy(best_weight_path, pt_dest)
    print(f"✅ Saved .pt weights to: {pt_dest}")

    # Export to TFLite
    print("Exporting to TFLite...")
    best_model = YOLO(best_weight_path)
    best_model.export(format='tflite')

    # Find the generated TFLite file (it might be in the same folder or a subdir)
    found_tflites = glob.glob(os.path.join(latest_run_dir, '**', '*.tflite'), recursive=True)

    if found_tflites:
        # Get the newest TFLite file just in case
        source_tflite = max(found_tflites, key=os.path.getmtime)

        tflite_dest = os.path.join(yolo_save_dir, 'yolo_hood_best.tflite')
        shutil.copy(source_tflite, tflite_dest)
        print(f"✅ Saved TFLite model to: {tflite_dest}")
    else:
        print("❌ Error: TFLite export failed or file not found.")
else:
    print(f"❌ Error: 'best.pt' missing in {latest_run_dir}")


--- Training YOLOv11 with Live Precision/Recall ---
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset, degrees=180, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11m-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=hood_run10, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask

In [ ]:
print("Exporting to OpenVINO...")
best_model = YOLO(best_weight_path)

# Export args:
# half=True (FP16 quantization, faster on modern CPUs/GPUs, slight acc loss)
# int8=True (INT8 quantization, much faster, requires valid data for calibration)
openvino_dir = best_model.export(format='openvino', half=True)

print(f"Exported to: {openvino_dir}")

# --- Zip and Save to Drive ---
if os.path.exists(openvino_dir):
    # Define zip name
    zip_name = "yolo_hood_openvino"
    zip_path = os.path.join(yolo_save_dir, zip_name)

    # Create zip file (shutil.make_archive adds .zip automatically)
    shutil.make_archive(zip_path, 'zip', openvino_dir)

    print(f"✅ OpenVINO model zipped and saved to: {zip_path}.zip")
else:
    print("❌ Error: OpenVINO export directory not found.")